- main (global policy and value networks)
    - Create and coordinate workers 
    - check CPUs available, create threads and workers
    - Initialize global thread safe counter,every workers know when to quit
    
- worker (contains local policy and value networks)
    - copy weights from global network
    - play episodes
    - send gradients back to master

\begin{equation*}
g_{local} = \frac {\partial L(\theta_{local})}{\partial \theta_{local}} \\
\theta_{global} = \theta_{global} - \eta g_{local}\\
\end{equation*}
    
- nets
    - definition of poilicy and value network
    - variable scopes
    - "reuse" arguments

In [1]:
import tensorflow as tf

In [ ]:
class PolicyNetwork:
    def __init__(self, num_outputs, reg=0.01):
        
        self.num_outputs = num_outputs
        
        self.states = tf.placeholder(shape=[None, 84, 84, 4], dtype=tf.uint8, name="X")
        
        # Advantage = G - V(s)
        self.advantage = tf.placeholder(shape=[None], dtype=tf.float32, name="y")
        
        # Selected actions
        self.actions = tf.placeholder(shape=[None], dtype=tf.int32, name="actions")
        
        # Since we set reuse=False here, that means we MUST
        # create the PolicyNetwork before creating the ValueNetwork
        # ValueNetwork will use reuse=True
        with tf.variable_scope("shared", reuse=False):
            fc1 = build_feature_extractor(self.states)
        
        # Use a separate scope for output and loss
        with tf.variable_scope("policy_network"):
            self.logits = tf.contrib.layers.fully_connected(fc1, num_outputs, activation_fn=None)
            self.probs = tf.nn.softmax(self.logits)
            
            # Sample an action
            cdist = tf.distributions.Categorical(logits=self.logits)
            self.sample_action = cdist.sample()
            
            # Add regularization to increase exploration
            self.entropy = -tf.reduce_sum(self.probs * tf.log(self.probs), axis=1)
            
            # Get the predictions for the chosen actions only
            batch_size = tf.shape(self.states)[0]
            gather_indices = tf.range(batch_size) * tf.shape(self.probs)[1] + self.actions
            self.selected_action_probs = tf.gather(tf.reshape(self.probs, [-1]), gather_indices)


### Scratch

In [2]:
sess = tf.InteractiveSession()

In [29]:
import numpy as np
cdist = tf.distributions.Categorical(logits=np.log([0.6,0.1,0.4]))
sess.run(cdist.sample())

0

In [38]:
batch_size = 5
outrange = sess.run(tf.range(batch_size))
outrange.shape

(5,)

In [53]:
X = tf.placeholder(tf.float32,[None,4])
Y = tf.placeholder(tf.float32,[None,3])

W = tf.Variable(tf.zeros([4,3]),name='weight')
b = tf.Variable(tf.zeros([3]),name='bias')
H = tf.nn.softmax(tf.matmul(X,W)+b)

# sess.run(H.eval())
# sess.run(tf.constant([0,1,2,3]))

sess.run(tf.shape(tf.nn.softmax(tf.constant([0.1,0.3,0.6])))[0])


3

In [59]:
actions = tf.constant([0,1,2,3,4])
gather_indices = tf.range(5) * tf.shape(tf.zeros([2,3]))[1] #+ actions
sess.run(gather_indices)

array([ 0,  3,  6,  9, 12], dtype=int32)